# Tutorial 0: The Core of pykal

## Overview

This tutorial will walk you through the genesis and structure of the pykal_core library. Upon finishing this tutorial, you will be able to model systems using the pykal_core library, as shown in tutorial 1. [link] You should also be able to move onto modeling a turtlebot, as shown in tutorial 1 [link].

This is the only tutorial in which the pronoun "I" is used instead of the more acedemic "we", as I found it strange to hide behind plurality when discussing design decisions whose benefits and shortcomings are ultimately my fault.

## The Feedback System

A basic feedback system[link] is shown below. 

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/system_input_observer_controller_block_diagram.svg" alt="System Observer Controller Block Diagram" style="width: 100%;">
   </div>

There are five distinct objects in this block diagram:

- Signal generator
- t (Signal transform)
- Dynamical System (or Plant)
- Controller
- Observer (or Estimator)

As a concrete example, consider the cruise control in your car. Recall that the goal of cruise control is to maintain a speed set by the driver. 

- the setpoint signal is generated when you set the speed using a button ( or if you're fancy, a touchscreen display)
- the difference between this setpoint and the current estimated speed is computed and sent to the controller
- the dynamical system is the car itself, where $y$ is the speedometer readings
- the controller looks at this difference and sends an appropriate signal to the car to either speed up or slow down (the controller abstracts away such things as gas pedal and brake pedal depression)
- the observer takes the controller signal and the speedometer's current readings and estimates what the current speed of the car is



### The Feedback System as Classes 

The control system objects are modeled by the following classes:

- Signal Generator (`Signal`)
- Signal Transformer (`Signal`)  
- Plant (`System`)
- Controller (`Controller`)
- Observer (`Observer`)

  
Note that in the feedback diagram above, the signal generator and signal transformer do not depend explicitly on the control system objects; rather, they just define or modify the signals passed from one block to another. To reflect this programmatically, the `Signal` class has no dependencies on the other classes. 

However, it doesn't make sense to have an observer or a controller without an underlying system. To reflect this, the `System` class is injected as a dependency into the `Observer` and `Controller` classes. 

Below, we get a feel for constructing these objects; a thorough discussion on each class and its usage will be provided in the upcoming section.

In [13]:
import pykal_core
from pykal_core.control_system import Signal, Controller, System, Observer

sig = Signal() # Signal constructor requires no arguments

sys = System(state_names=["x0","x1"]) # state names are the only required argument for the System constructor

obs = Observer(sys) # a System object must be passed into the Observer and Controller constructors 
cont = Controller(sys)

print(obs.sys) # the Observer and Controller objects reference the same System object
print(cont.sys)

In the following sections, we will discuss the aforementioned classes in the `control_system` module. For each class, the control theoretic conceptual primitive will be introduced and the mapping into each class will be explained. Basic attributes and usage of each class will also be explained, with links provided for further reading when appropriate. 

## Control System

### Signal

#### Attributes

`sig.generate` generates signal functions based on certain params.

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/signal_generator_block.svg" alt="System Observer Controller Block Diagram" style="width: 30%;">
     <img src="../../_static/signals_heaviside_and_sin.svg" style="width: 70%;">     
   </div>

In [14]:
from pykal_core.control_system import Signal

sig = Signal() # Signal constructor requires no arguments

unit_step_func = sig.generate.heaviside_step_function(output=[1],step_time=5) # generate signal functions with params
sin_wave_func = sig.generate.sinusoidal_function(amplitudes=[1],frequencies=[0.1])

from pykal_core.utils.general import Simulation

unit_step_df = Simulation.of_signals(signal_generator=unit_step_func,
                      t_span = (0,10),
                      dt = 0.1,
                      output_df=True                      
                      )

sin_wave_df = Simulation.of_signals(signal_generator=sin_wave_func,
                      t_span = (0,10),
                      dt = 0.1,
                      output_df=True
                      )

from matplotlib import pyplot as plt 

fig, axes = plt.subplots(1, 2, figsize=(10, 4),sharey=True)

# Unit step
axes[0].plot(unit_step_df.index, unit_step_df.iloc[:, 0],color="blue")
axes[0].set_title("Heaviside Step")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Signal Value")

# Sine wave
axes[1].plot(sin_wave_df.index, sin_wave_df.iloc[:, 0],color="orange")
axes[1].set_title("Sinusoidal Signal")
axes[1].set_xlabel("Time (s)")

plt.tight_layout()

plt.savefig("../../_static/signals_heaviside_and_sin.svg", format="svg",transparent=True) # save as SVG

#plt.show()
plt.close(fig)

`sig.transform` has unary transformations (denoted in a block diagram by a triangle) that can perform things such as scaling or inverting signals (or the user can define their own transformation, as seen here)

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/signal_transform_block_unary.svg" alt="System Observer Controller Block Diagram" style="width: 20%;">
      <img src="../../_static/signals_inverted_heaviside.svg" style="width: 80%;">     
   </div>

In [15]:
from pykal_core.control_system import Signal

sig = Signal() # Signal constructor requires no arguments

unit_step_func = sig.generate.heaviside_step_function(output=[1],step_time=5) # generate signal functions with params

inverted_unit_step_func = sig.transform.signals(sig.transform.negate,unit_step_func)

from pykal_core.utils.general import Simulation

unit_step_df = Simulation.of_signals(signal_generator=unit_step_func,
                      t_span = (0,10),
                      dt = 0.1,
                      output_df=True                      
                      )

inverted_unit_step_df = Simulation.of_signals(signal_generator=inverted_unit_step_func,
                      t_span = (0,10),
                      dt = 0.1,
                      output_df=True
                      )

from matplotlib import pyplot as plt 

fig, axes = plt.subplots(1, 2, figsize=(10, 4),sharey=True)

# Unit step
axes[0].plot(unit_step_df.index, unit_step_df.iloc[:, 0],color="blue")
axes[0].set_title("Heaviside Step")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Signal Value")

# Sine wave
axes[1].plot(inverted_unit_step_df.index, inverted_unit_step_df.iloc[:, 0],color="blue")
axes[1].set_title("Inverted Heaviside Signal")
axes[1].set_xlabel("Time (s)")

plt.tight_layout()

plt.savefig("../../_static/signals_inverted_heaviside.svg", format="svg",transparent=True) # save as SVG

#plt.show()
plt.close(fig)

`sig.transform` has n-nary transformations (denoted in a block diagram by a circle) that can perform transformations on n-many signals such as taking the product, taking the difference, or a user defined transformation (as seen here)

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/signal_transform_block_nnary.svg" alt="System Observer Controller Block Diagram" style="width: 20%;">
     <img src="../../_static/signals_heaviside_and_sin_product.svg" alt="System Observer Controller Block Diagram" style="width: 80%;">
   </div>

Include figure for unary vs binary transforms with examples (signal inversion vs joining signals). Also include figure for G, and then triangle for unary and circle for n-ary

In [16]:
from pykal_core.control_system import Signal

sig = Signal() # Signal constructor requires no arguments

unit_step_func = sig.generate.heaviside_step_function(output=[1],step_time=5) # generate signal functions with params
sin_wave_func = sig.generate.sinusoidal_function(amplitudes=[1],frequencies=[0.1])

# create a new signal function by applying a transform to two signal functions
sin_wave_func_times_unit_step_func = sig.transform.signals(sig.transform.multiply,sin_wave_func,unit_step_func)


from pykal_core.utils.general import Simulation

unit_step_df = Simulation.of_signals(signal_generator=unit_step_func,
                      t_span = (0,10),
                      dt = 0.1,
                      output_df=True                      
                      )

sin_wave_df = Simulation.of_signals(signal_generator=sin_wave_func,
                      t_span = (0,10),
                      dt = 0.1,
                      output_df=True
                      )

sin_wave_times_unit_step_df = Simulation.of_signals(signal_generator=sin_wave_func_times_unit_step_func,
                      t_span = (0,10),
                      dt = 0.1,
                      output_df=True
                      )

from matplotlib import pyplot as plt 

fig, axes = plt.subplots(1, 3, figsize=(10, 4),sharey=True)

# Unit step
axes[0].plot(unit_step_df.index, unit_step_df.iloc[:, 0],color="blue")
axes[0].set_title("Heaviside Step")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Signal Value")

# Sine wave
axes[1].plot(sin_wave_df.index, sin_wave_df.iloc[:, 0],color="orange")
axes[1].set_title("Sinusoidal Signal")
axes[1].set_xlabel("Time (s)")

# sin_wave_times_unit_step_df wave
axes[2].plot(sin_wave_times_unit_step_df.index, sin_wave_times_unit_step_df.iloc[:, 0],color="green")
axes[2].set_title("Product")
axes[2].set_xlabel("Time (s)")

plt.tight_layout()

plt.savefig("../../_static/signals_heaviside_and_sin_product.svg", format="svg",transparent=True) # save as SVG

#plt.show()
plt.close(fig)


#### Usage

### System

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/system_block_diagram.svg" alt="System Observer Controller Block Diagram" style="width: 100%;">
   </div>

Recall that the state-space model of a continuous-time dynamical system is given by:

$$
\begin{aligned}
\dot{x} &= f(x, u, t) + w(t) \\
y &= h(x, u, t) + r(t)
\end{aligned}
$$

Where:

- $x \in \mathbb{R}^{n}$ is the **state vector**
- $u \in \mathbb{R}^{p}$ is the **input vector**
- $t \in \mathbb{R}^{+}$ is **time**
- $w(t) \in \mathbb{R}^{n}$ is **process noise**, distributed as $w \sim \mathcal{N}(0, Q(t))$, where $Q(t) \in \mathbb{R}^{n \times n}$ is the **process noise covariance matrix**

Similarly:

- $y \in \mathbb{R}^{m}$ is the **measurement vector**
- $r(t) \in \mathbb{R}^{m}$ is **measurement noise**, distributed as $r \sim \mathcal{N}(0, R(t))$, where $R(t) \in \mathbb{R}^{m \times m}$ is the **measurement noise covariance matrix**

In [17]:
from pykal_core.control_system import System

sys = System(state_names=["x0","x1"]) # state names are the only required argument for the constructor

for key, value in sys.__dict__.items():
    print(f"{key}: {value}")

safeio: <pykal_core.utils.system.safeio.SafeIO object at 0x717f58e2d880>
_state_names: ['x0', 'x1']
_measurement_names: ['x0_meas', 'x1_meas']
_system_type: cti
_f: <function System.f_zero at 0x717fc0171e40>
_h: <function System.h_identity at 0x717f5ec95a80>
_Q: <function System.make_Q.<locals>.Q at 0x717f589b1260>
_R: <function System.make_R.<locals>.R at 0x717f589b0b80>


Note that the `sys` object is initialized with several attributes if they are not passed explicity by the constructor (the `System.safeio` attribute is an instance of a utility class called `SafeIO` and is discussed here [link]). The default values of each attribute is described briefly in the "System Attributes" subsection. 

#### Attributes

`sys.measurement_names` are the default measurement names, which are are just '_meas' appended to each state name

In [18]:
sys.measurement_names

['x0_meas', 'x1_meas']

`sys.system_type` is the time structure of the dynamical system (default is "cti", or "continuous time-invariant")

In [19]:
sys.system_types # all available system time structures for sys

{'cti', 'ctv', 'dti', 'dtv'}

`System.f_zero` is the function $f(x,u,t) = 0$, that is, our plant has zero dynamics and does not change over time.

In [20]:
import numpy as np
xk = np.array([[1.0], [0.0]])
sys.f(xk)

array([[0.],
       [0.]])

`System.h_identity` is the function $h(x,u,t)=x$ ie the measurements are simply the state.

In [21]:
xk = np.array([[1.0], [0.0]])
sys.h(xk)

array([[1.],
       [0.]])

`System.make_Q.<locals>.Q` is the process noise covariance $Q$ created by a factory function `System.make_Q`. The default $Q$ returns an identity matrix with noise $0.01$ on the diagonals.    

In [22]:
sys.Q()

array([[0.01, 0.  ],
       [0.  , 0.01]])

`System.make_R.<locals>.R` is the measurement noise covariance $R$ created by a factory function `System.make_R`. The default $R$ returns an identity matrix with noise $0.01$ on the diagonals.

In [23]:
sys.R()

array([[0.01, 0.  ],
       [0.  , 0.01]])

#### Usage

### Observer

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/system_input_observer_block_diagram.svg" alt="System Observer Controller Block Diagram" style="width: 100%;">
   </div>

Recall that the observer of a dynamical system is given by:

$$
\hat{x} = L(u,y)
$$

Where:
- $\hat{x} \in \mathbb{R}^{n}$ is the **estimated state vector**
- $y \in \mathbb{R}^{m}$ is the **measurement vector**
- $u \in \mathbb{R}^{p}$ is the **input vector**

Note that there is only one function that define the behavior of the observer: $L$. But this is high-level, often complexity comes in the functions composing L. For example, the kalman filter only requires y and u as active inputs, but is initialized with ..., and internal stuff.

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/observer_kf.svg" alt="Observer kf  Diagram" style="width: 50%;">
   </div>

In [24]:
from pykal_core.control_system.observer import Observer
obs = Observer(sys)

ModuleNotFoundError: No module named 'pykal_core.control_system.observer'; 'pykal_core.control_system' is not a package

In [ ]:
for key, value in obs.__dict__.items():
    print(f"{key}: {value}")

sys: <pykal_core.control_system.system.System object at 0x7b5ac8998920>
L: None


In [ ]:
from pykal_core.utils.estimators import kf
obs.L = kf.make_L(obs,
                  kf.predict_EKF,
                  kf.update_EKF)

In [ ]:
obs.L.__name__

'kf'

#### Attributes

#### Usage

For all estimators and how to use hhem, see the API reference here:

### Controller

   <div style="display: flex; align-items: center; justify-content: space-between; gap: 2em; margin-top: 1em; margin-bottom: 1em;">
     <img src="../../_static/system_input_observer_controller_block_diagram.svg" alt="System Observer Controller Block Diagram" style="width: 100%;">
   </div>

#### Attributes

#### Usage